## Assignmeent-3 MA24M005(Biswajit Gorai)

---


### Fine-tuning the a ASR Model (whisper-tiny)

In [1]:
# neceesary libary installation
!pip install datasets==3.6.0 transformers evaluate jiwer

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 491.5/491.5 kB 11.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 4.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.2/3.2 MB 25.8 MB/s eta 0:00:00
  Attempting uninstall: datasets
    Found existing installation: datasets 4.0.0
    Uninstalling datasets-4.0.0:
      Successfully uninstalled datasets-4.0.0


In [2]:
import torch
import numpy as np
from datasets import load_dataset, Audio
from transformers import WhisperProcessor, WhisperFeatureExtractor, WhisperTokenizer, WhisperForConditionalGeneration
from transformers import Seq2SeqTrainer, Seq2SeqTrainingArguments
import evaluate
from dataclasses import dataclass
from typing import Any, Dict, List, Union


In [3]:
from datasets import load_dataset, DatasetDict
hindi_voice=DatasetDict()

hindi_voice['train']=load_dataset("mozilla-foundation/common_voice_11_0", "hi",split="train[:1100]")
hindi_voice['test']=load_dataset("mozilla-foundation/common_voice_11_0", "hi",split='test[:10]')

print(hindi_voice)

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md: 0.00B [00:00, ?B/s]

common_voice_11_0.py: 0.00B [00:00, ?B/s]

languages.py: 0.00B [00:00, ?B/s]

release_stats.py: 0.00B [00:00, ?B/s]

...

Generating other split: 0 examples [00:00, ? examples/s]


Reading metadata...: 3328it [00:00, 101012.71it/s]


Generating invalidated split: 0 examples [00:00, ? examples/s]


Reading metadata...: 680it [00:00, 52541.80it/s]


DatasetDict({
    train: Dataset({
        features: ['client_id', 'path', 'audio', 'sentence', 'up_votes', 'down_votes', 'age', 'gender', 'accent', 'locale', 'segment'],
        num_rows: 1100
    })
    test: Dataset({
        features: ['client_id', 'path', 'audio', 'sentence', 'up_votes', 'down_votes', 'age', 'gender', 'accent', 'locale', 'segment'],
        num_rows: 10
    })
})


### prepare feature extractor, tokenizer and data

In [4]:
from transformers import  WhisperFeatureExtractor

feature_extractor=WhisperFeatureExtractor.from_pretrained("openai/whisper-tiny")

preprocessor_config.json: 0.00B [00:00, ?B/s]

In [5]:
from transformers import  WhisperTokenizer

tokenizer=WhisperTokenizer.from_pretrained("openai/whisper-tiny",langauge='hindi',task='transcribe')

tokenizer_config.json: 0.00B [00:00, ?B/s]

vocab.json: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

merges.txt: 0.00B [00:00, ?B/s]

normalizer.json: 0.00B [00:00, ?B/s]

added_tokens.json: 0.00B [00:00, ?B/s]

special_tokens_map.json: 0.00B [00:00, ?B/s]

In [6]:
from transformers import  WhisperProcessor
processor = WhisperProcessor.from_pretrained(
    "openai/whisper-tiny",
    language="Hindi",
    task="transcribe"
)

In [7]:
print(hindi_voice['train'][0])

{'client_id': '0f018a99663f33afbb7d38aee281fb1afcfd07f9e7acd00383f604e1e17c38d6ed8adf1bd2ccbf927a52c5adefb8ac4b158ce27a7c2ed9581e71202eb302dfb3', 'path': '/root/.cache/huggingface/datasets/downloads/extracted/1bfc12b9ee30f73bf143fa237d4ba38488008883c25816876e1a35295c9575d3/hi_train_0/common_voice_hi_26008353.mp3', 'audio': {'path': '/root/.cache/huggingface/datasets/downloads/extracted/1bfc12b9ee30f73bf143fa237d4ba38488008883c25816876e1a35295c9575d3/hi_train_0/common_voice_hi_26008353.mp3', 'array': array([ 5.81611368e-26, -1.48634016e-25, -9.37040538e-26, ...,
        1.06425901e-07,  4.46416450e-08,  2.61450239e-09]), 'sampling_rate': 48000}, 'sentence': 'हमने उसका जन्मदिन मनाया।', 'up_votes': 2, 'down_votes': 0, 'age': '', 'gender': '', 'accent': '', 'locale': 'hi', 'segment': ''}


In [8]:
from datasets import Audio
hindi_voice=hindi_voice.cast_column('audio',Audio(sampling_rate=16000))

In [9]:
print(hindi_voice['train'][0])

{'client_id': '0f018a99663f33afbb7d38aee281fb1afcfd07f9e7acd00383f604e1e17c38d6ed8adf1bd2ccbf927a52c5adefb8ac4b158ce27a7c2ed9581e71202eb302dfb3', 'path': '/root/.cache/huggingface/datasets/downloads/extracted/1bfc12b9ee30f73bf143fa237d4ba38488008883c25816876e1a35295c9575d3/hi_train_0/common_voice_hi_26008353.mp3', 'audio': {'path': '/root/.cache/huggingface/datasets/downloads/extracted/1bfc12b9ee30f73bf143fa237d4ba38488008883c25816876e1a35295c9575d3/hi_train_0/common_voice_hi_26008353.mp3', 'array': array([ 3.81639165e-17,  2.42861287e-17, -1.73472348e-17, ...,
       -1.30981789e-07,  2.63096808e-07,  4.77157300e-08]), 'sampling_rate': 16000}, 'sentence': 'हमने उसका जन्मदिन मनाया।', 'up_votes': 2, 'down_votes': 0, 'age': '', 'gender': '', 'accent': '', 'locale': 'hi', 'segment': ''}


In [10]:
def prepare_dataset(batch):
  audio=batch['audio']

  batch['input_features']=feature_extractor(audio['array'],sampling_rate=audio['sampling_rate']).input_features[0]

  batch['labels']=tokenizer(batch['sentence']).input_ids
  return batch

In [11]:
processed_train = hindi_voice.map(prepare_dataset,remove_columns=hindi_voice.column_names['train'],num_proc=1)


Map:   0%|          | 0/1100 [00:00<?, ? examples/s]

Map:   0%|          | 0/10 [00:00<?, ? examples/s]

In [12]:
processed_test=hindi_voice.map(prepare_dataset,remove_columns=hindi_voice.column_names['test'],num_proc=1)

In [13]:
train_data=hindi_voice['train']
test_data=hindi_voice['test']

In [14]:
from transformers import WhisperForConditionalGeneration

model = WhisperForConditionalGeneration.from_pretrained("openai/whisper-tiny")
model.config.forced_decoder_ids = None
model.config.suppress_tokens = []


config.json: 0.00B [00:00, ?B/s]

model.safetensors:   0%|          | 0.00/151M [00:00<?, ?B/s]

generation_config.json: 0.00B [00:00, ?B/s]

In [16]:
@dataclass
class DataCollatorSpeechSeq2SeqWithPadding:
    processor: Any
    def __call__(self, features):
        input_features = [{"input_features": f["input_features"]} for f in features]
        label_features = [{"input_ids": f["labels"]} for f in features]
        batch = self.processor.feature_extractor.pad(input_features, return_tensors="pt")
        labels_batch = self.processor.tokenizer.pad(label_features, return_tensors="pt")
        batch["labels"] = labels_batch["input_ids"].masked_fill(labels_batch["attention_mask"].ne(1), -100)
        return batch

data_collator = DataCollatorSpeechSeq2SeqWithPadding(processor=processor)


In [17]:
import evaluate
wer_metric = evaluate.load("wer")

def compute_metrics(pred):
    pred_ids = pred.predictions
    label_ids = pred.label_ids
    pred_str = tokenizer.batch_decode(pred_ids, skip_special_tokens=True)
    label_str = tokenizer.batch_decode(label_ids, skip_special_tokens=True)
    wer = wer_metric.compute(predictions=pred_str, references=label_str)
    return {"wer": wer}


In [18]:
from transformers import Seq2SeqTrainingArguments

training_args = Seq2SeqTrainingArguments(
    output_dir="./whisper-hi",
    report_to="none",
    per_device_train_batch_size=8,
    per_device_eval_batch_size=1,
    warmup_steps=50,
    max_steps=100,
    save_steps=100,
    eval_steps=100,
    logging_steps=10,
    eval_strategy="steps",
    predict_with_generate=True,
)


In [19]:
print(processed_train)

DatasetDict({
    train: Dataset({
        features: ['input_features', 'labels'],
        num_rows: 1100
    })
    test: Dataset({
        features: ['input_features', 'labels'],
        num_rows: 10
    })
})


In [20]:
from transformers import Seq2SeqTrainer

trainer = Seq2SeqTrainer(
    model=model,
    args=training_args,
    train_dataset=processed_train['train'],
    eval_dataset=processed_train['test'],
    data_collator=data_collator,
    compute_metrics=compute_metrics,
    tokenizer=processor.feature_extractor,
)
trainer.train()


/tmp/ipython-input-675103295.py:3: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Seq2SeqTrainer.__init__`. Use `processing_class` instead.
  trainer = Seq2SeqTrainer(
You're using a WhisperTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


Step,Training Loss,Validation Loss,Wer
100,0.716400,0.860183,0.767442


Using custom `forced_decoder_ids` from the (generation) config. This is deprecated in favor of the `task` and `language` flags/config options.
Transcription using a multilingual Whisper will default to language detection followed by transcription instead of translation to English. This might be a breaking change for your use case. If you want to instead always translate your audio to English, make sure to pass `language='en'`. See https://github.com/huggingface/transformers/pull/28687 for more details.
The attention mask is not set and cannot be inferred from input because pad token is same as eos token. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
/usr/local/lib/python3.12/dist-packages/transformers/modeling_utils.py:3918: UserWarning: Moving the following attributes in the config to the generation config: {'max_length': 448, 'suppress_tokens': [], 'begin_suppress_tokens': [220, 50257]}. You are seeing thi

TrainOutput(global_step=100, training_loss=1.2380027151107789, metrics={'train_runtime': 181.313, 'train_samples_per_second': 4.412, 'train_steps_per_second': 0.552, 'total_flos': 1.9695108096e+16, 'train_loss': 1.2380027151107789, 'epoch': 0.7246376811594203})

In [21]:
metrics = trainer.evaluate()
print(metrics)


{'eval_loss': 0.8601834177970886, 'eval_wer': 0.7790697674418605, 'eval_runtime': 4.6588, 'eval_samples_per_second': 2.146, 'eval_steps_per_second': 2.146, 'epoch': 0.7246376811594203}
